In [1]:
import os
import torch
import librosa
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve

/home/manish/miniconda3/envs/speech/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch.nn as nn

In [3]:
#GPU device
device = 'cuda' if torch.cuda.is_available() else 'cpu'                  
print('Device: {}'.format(device))

# Load the SSL W2V model trained for LA and DF tracks
from model import Model

model = Model(None, device=device)
model = nn.DataParallel(model).to(device)
model.load_state_dict(torch.load('/mnt/c/Users/Manish/Desktop/SpeechAssign/SSL_Anti-spoofing/model.pth'))

model.eval()

Device: cuda


: 

In [ ]:
def pad(x, max_len=64600):
    x_len = x.shape[0]
    if x_len >= max_len:
        return x[:max_len]
    # need to pad
    num_repeats = int(max_len / x_len)+1
    padded_x = np.tile(x, (1, num_repeats))[:, :max_len][0]
    return padded_x	

# Define a function to preprocess the audio samples
def preprocess_audio(audio_path):
    # Check file extension
    _, ext = os.path.splitext(audio_path)
    if ext.lower() not in ('.mp3', '.wav'):
        # Skip processing if file extension is not .mp3 or .wav
        return None
    
    # Load the audio file and extract features
    audio, sr = librosa.load(audio_path, sr=None)
    audio = pad(audio)
    # Here you can apply further preprocessing if needed, e.g., feature extraction
    return audio


# Define paths to real and fake audio samples
real_audio_dir = r"/mnt/c/Users/Manish/Desktop/SpeechAssign/SSL_Anti-spoofing/Dataset1/Dataset_Speech_Assignment/Real"
fake_audio_dir = r"/mnt/c/Users/Manish/Desktop/SpeechAssign/SSL_Anti-spoofing/Dataset1/Dataset_Speech_Assignment/Fake"

# Collect paths to real and fake audio files
real_audio_paths = [os.path.join(real_audio_dir, filename) for filename in os.listdir(real_audio_dir)]
fake_audio_paths = [os.path.join(fake_audio_dir, filename) for filename in os.listdir(fake_audio_dir)]

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
import numpy as np
from tqdm import tqdm

# Collect predictions and ground truth labels
predictions = []
ground_truth = []

# Use tqdm to add a progress bar
for audio_path in tqdm(real_audio_paths + fake_audio_paths):
    processed_audio = preprocess_audio(audio_path)
    
    # Skip processing if preprocess_audio returns None
    if processed_audio is None:
        continue
    
    processed_audio_tensor = torch.tensor(processed_audio, dtype=torch.float32).unsqueeze(0)
    print(processed_audio_tensor.shape)
    
    with torch.no_grad():
        output = model(processed_audio_tensor)
        # Move the tensor from CUDA device to CPU
        output_cpu = output.cpu()

        # Extract the second value (index 1) and convert it to a Python scalar
        prediction = output_cpu[0][1].item()
        
    predictions.append(prediction)
    
    # Add ground truth label
    if audio_path in real_audio_paths:
        ground_truth.append(0)  # 0 for real
    else:
        ground_truth.append(1)  # 1 for fake

In [ ]:
# Calculate AUC
auc_score = roc_auc_score(ground_truth, predictions)

# Calculate ROC curve
fpr, tpr, _ = roc_curve(ground_truth, predictions)

# Find the point on the ROC curve where FPR equals 1 - TPR
eer = 1.0
for i in range(len(fpr)):
    if fpr[i] >= 1 - tpr[i]:
        eer = fpr[i]
        break

# Analyze the performance of the model
print("AUC:", auc_score)
print("EER:", eer)